### Import librairies
Nous importons les bibliothèque fondamentale pour la programme

In [1]:
import numpy as np
import pandas as pd

###  Load Data
Nous allons importer les données des trainSet et testSet dans les features et la variable à déviner

In [2]:
path='../3-DataSet/dataFinale/p25/testSet_p25.csv'
#path='../3-DataSet/dataFinale/p27/testSet_p27.csv'
#path='../3-DataSet/dataFinale/p230/testSet_p30.csv'

#cols=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4"]
cols=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
toPredict = pd.read_csv(path,names=cols, sep=',')
toPredict.head(5)

,ECP,RC,MIN,MAX,MEAN,START,END,DURATION,CV,A1,A2,A3,A4,FP
0,3005FB63AC1F3681EC8815B921,9,-58.5,-52.0,-56.555556,1581420973596918,1581420974726948,1130030,-0.040071,3,0,6,0,0
1,E2806890000000001834376767,2,-63.0,-62.0,-62.500000,1581427757606159,1581427757711480,105321,-0.008000,0,0,2,0,0
2,E2806890000000001834531777,66,-63.5,-58.0,-60.045455,1581426772161856,1581426781271598,9109742,-0.016888,66,0,0,0,1
3,2005FB63EAC13681E4EC0FD224,4,-65.0,-59.0,-62.000000,1581420375415960,1581420375804651,388691,-0.038253,0,4,0,0,0
4,3005FB63AC1F3681EC88142830,7,-57.5,-45.0,-50.928571,1581414771515647,1581414773005717,1490070,-0.087408,7,0,0,0,0


### Normalisation et mise en echelle
Avant de faire des prédictions réelles, il est toujours recommandé de mettre à l'échelle les entités afin que toutes puissent être évaluées uniformément. 
L'algorithme de descente de gradient (qui est utilisé dans la formation de réseaux de neurones et d'autres algorithmes d'apprentissage automatique) converge également plus rapidement avec des fonctionnalités normalisées.
![](scaling.JPG)

In [3]:
from sklearn.preprocessing import StandardScaler

def scaleData(data):
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

In [5]:
X_toPredict = scaleData(toPredict.loc[:,'RC':'A4'])
y_toPredict = toPredict['FP']

In [14]:
X=scaleData(toPredict.loc[:,'RC':'A4'])
xx = scaleData(X)
xx[0]

array([ 0.39828109,  0.36130347,  0.86928704,  0.3422276 , -0.31300398,
       -0.31293927,  0.21437791, -0.67721954,  0.01208223, -0.61705151,
        2.57133008, -0.17637297])

In [11]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Contrive small dataset
dataset = [[50, 30], [20, 90]]
#print(dataset)
# Calculate min and max for each column
minmax = dataset_minmax(X)
#print(minmax)
# Normalize columns
normalize_dataset(X, minmax)
print(X[0])

[0.12121212 0.64583333 0.73333333 0.69430625 0.43546632 0.43538883
 0.12404632 0.63231083 0.04477612 0.         0.5        0.        ]


In [51]:
from math import sqrt
 
# calculate column means
def column_means(dataset):
    #means = [0 for i in range(len(dataset[0]))]
    mean=[]
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        #means[i] = sum(col_values) / float(len(dataset))
        s= (sum(col_values) / float(len(dataset)))
        means.append(s)
    return means
 
# calculate column standard deviations
def column_stdevs(dataset, means):
    #stdevs = [0 for i in range(len(dataset[0]))]
    stdevs=[]
    for i in range(len(dataset[0])):
        variance = [pow(row[i]-means[i], 2) for row in dataset]
        #stdevs[i] = sum(variance)
        stdevs.append(sum(variance))
    s = [sqrt(x/(float(len(dataset)-1))) for x in stdevs]
    stdevs.append(s)
    return stdevs
 
# standardize dataset
def standardize_dataset(dataset, means, stdevs):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - means[i]) / stdevs[i]
            #row.append((row[i] - means[i]) / stdevs[i])
            
 
#print(X)
# Estimate mean and standard deviation
means = column_means(X)
stdevs = column_stdevs(X, means)
#print(means)
#print(stdevs)
# standardize dataset
standardize_dataset(X, means, stdevs)
X=X.tolist()
print(X)

[[0.0017662176075088213, 0.001602236642799914, 0.003854940922640841, 0.0015176427731050089, -0.001388047683818545, -0.0013877607164876227, 0.0009506804466474913, -0.003003198230781716, 5.357988533318164e-05, -0.0027363770703543165, 0.011402822163634007, -0.0007821436934684167], [-0.0016621949754101397, -0.0037933748331190013, -0.005354537391490556, -0.005159356839682442, 0.004430680662803824, 0.004429958734622768, -0.002313735901124349, 0.0031324259627625983, -0.0013995066049026813, -0.0027363770703543165, 0.0021578434767056285, -0.0007821436934684167], [0.029683291496991795, -0.0043928872193322146, -0.0016707460658379966, -0.0024023289621848192, 0.0035854529374007866, 0.0035924729247183247, 0.02637165586826585, 0.001432110247588534, 0.03056839618028629, -0.0027363770703543165, -0.0024646458667585596, -0.0007821436934684167], [-0.0006826485231475799, -0.0061914243779718535, -0.002591693897251136, -0.004597740049821815, -0.0019011148647546457, -0.0019014522751993497, -0.0014110038469820

### Load models and Predict
Nous allons charger les différents models enregistrés puis predire les données avec chacun des models

In [ ]:
import pickle

#modelFilename = 'logreg_Model.sav'
#model = pickle.load(open(modelFilename, 'rb'))

modelFilename = 'knn_Model.sav'
model = pickle.load(open(modelFilename, 'rb'))

#modelFilename = 'gaussianSVC_Model.sav'
#model = pickle.load(open(modelFilename, 'rb'))

### Evaluation
Pour évaluer un algorithme, la matrice de confusion, la précision, le rappel et le score f1 sont les mesures les plus couramment utilisées. Les méthodes confusion_matrixet classification_reportde sklearn.metricspeuvent être utilisées pour calculer ces métriques. Jetez un œil au script suivant:

In [9]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

def showConfusionMatrix(ytest,ypred):
    print("Confusion Matrix \n",confusion_matrix(ytest, ypred))
    print("Repport: \n",classification_report(ytest, ypred))
    
def printMetrics(yTest, yPred):
    print("Accuracy: ", metrics.accuracy_score(yTest, yPred))
    print("Ajusted Rand Square",metrics.adjusted_rand_score(yTest, yPred))

### Prediction

In [ ]:
### Logistic Regression
#pred = model.predict(X_toPredict)

#showConfusionMatrix(y_toPredict,pred)
#printMetrics(y_toPredict, pred)

In [10]:
#Knn Predict
pred = model.predict(X_toPredict)

showConfusionMatrix(y_toPredict,pred)
printMetrics(y_toPredict, pred)

Confusion Matrix 
 [[217   1]
 [  2   6]]
Repport: 
              precision    recall  f1-score   support

          0       0.99      1.00      0.99       218
          1       0.86      0.75      0.80         8

avg / total       0.99      0.99      0.99       226

Accuracy:  0.986725663717
Ajusted Rand Square 0.781849563568


In [1]:
#SVM Gaussien
#pred = svmModel.predict(X_toPredict)

#showConfusionMatrix(y_toPredict,pred)
#printMetrics(y_toPredict, pred)

### Build DataSet

In [11]:
toPredict['IS_FP']=pred
print(toPredict.shape)
toPredict.tail(50)

(226, 15)


,ECP,RC,MIN,MAX,MEAN,START,END,DURATION,CV,A1,A2,A3,A4,FP,IS_FP
176,3005FB63AC1F3681EC88155C32,7,-63.0,-45.5,-51.285714,1581414921715666,1581414923139135,1423469,-0.102365,6,0,1,0,0,0
177,1532FB63AC1F3681EC8804686869,7,-60.0,-46.5,-52.214286,1581427723008583,1581427724502081,1493498,-0.082986,0,6,1,0,0,0
178,1007CEA7AC1F3681EC88046874,10,-61.5,-53.0,-57.050000,1581426442227620,1581426443994183,1766563,-0.060014,7,3,0,0,0,0
179,7336CEF1C1FE3682DEA1005C67,1,-57.5,-57.5,-57.500000,1581427757404342,1581427757404342,0,-0.000000,0,1,0,0,0,0
180,2005FB63EAC13681E4EC0F5070,5,-62.5,-58.5,-60.700000,1581427432556212,1581427434028015,1471803,-0.026869,0,4,1,0,0,0
181,2005FB63AC1F3681AB1004E373,10,-60.5,-53.5,-56.400000,1581426581988242,1581426583929006,1940764,-0.043213,0,0,10,0,0,0
182,7336CEF1C1FE3682DEA1005C33,4,-57.5,-56.5,-57.125000,1581414580770533,1581414581311679,541146,-0.007257,0,0,4,0,0,0
183,3005FB63AC1F3681EC8805AA25,3,-58.0,-54.5,-56.000000,1581420217820796,1581420219157190,1336394,-0.026285,2,0,1,0,0,0
184,E280689000000000183444A670,3,-65.5,-63.5,-64.666667,1581427432213108,1581427432944516,731408,-0.013142,3,0,0,0,0,0
185,3005FB63AC1F3681EC88145130,3,-54.0,-50.0,-52.000000,1581414772421872,1581414772990438,568566,-0.031404,3,0,0,0,0,0


### Evaluation

In [13]:
def rate(y, y_pred):
    """Return classification error of `(X, y)` with prediction `y_pred`."""
    return 100-((y != y_pred).sum() / len(y))

success = rate(toPredict['FP'], toPredict['IS_FP'])
print("Success rate: ",success)

Success rate:  99.9867256637


### Export

In [72]:
pathExport='../6-ClassifiedData/predctBySavedModel_25.csv'
toPredict.to_csv(pathExport, index = None, header=True)